<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introduction</a></span></li><li><span><a href="#Disclaimer" data-toc-modified-id="Disclaimer-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Disclaimer</a></span></li><li><span><a href="#Intended-Outcomes" data-toc-modified-id="Intended-Outcomes-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Intended Outcomes</a></span></li><li><span><a href="#Preparation:-System-Setup" data-toc-modified-id="Preparation:-System-Setup-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Preparation: System Setup</a></span><ul class="toc-item"><li><span><a href="#Install-the-IDE" data-toc-modified-id="Install-the-IDE-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Install the IDE</a></span></li><li><span><a href="#Install-the-opentrons-module" data-toc-modified-id="Install-the-opentrons-module-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Install the opentrons module</a></span></li><li><span><a href="#Keeping-the-opentrons-module-up-to-date" data-toc-modified-id="Keeping-the-opentrons-module-up-to-date-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Keeping the opentrons module up-to-date</a></span></li></ul></li><li><span><a href="#Setting-up-a-virtual-environment-for-simulating-a-robot" data-toc-modified-id="Setting-up-a-virtual-environment-for-simulating-a-robot-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Setting up a virtual environment for simulating a robot</a></span></li><li><span><a href="#Labware-objects" data-toc-modified-id="Labware-objects-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Labware objects</a></span><ul class="toc-item"><li><span><a href="#Putting-labware-objects-on-the-deck" data-toc-modified-id="Putting-labware-objects-on-the-deck-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Putting labware objects on the deck</a></span></li><li><span><a href="#Getting-wells-within-a-labware" data-toc-modified-id="Getting-wells-within-a-labware-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>Getting wells within a labware</a></span></li></ul></li><li><span><a href="#Setting-up-pipettes" data-toc-modified-id="Setting-up-pipettes-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Setting up pipettes</a></span></li><li><span><a href="#Getting-readouts-from-the-console" data-toc-modified-id="Getting-readouts-from-the-console-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Getting readouts from the console</a></span></li><li><span><a href="#Executing-a-simple-liquid-transfer" data-toc-modified-id="Executing-a-simple-liquid-transfer-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Executing a simple liquid transfer</a></span><ul class="toc-item"><li><span><a href="#&quot;one-to-one&quot;-transfer" data-toc-modified-id="&quot;one-to-one&quot;-transfer-9.1"><span class="toc-item-num">9.1&nbsp;&nbsp;</span>"one-to-one" transfer</a></span></li><li><span><a href="#Large-volumes" data-toc-modified-id="Large-volumes-9.2"><span class="toc-item-num">9.2&nbsp;&nbsp;</span>Large volumes</a></span></li><li><span><a href="#Scaling-up-transfer-by-the-&quot;one-to-many&quot;,-&quot;many-to-one&quot;,-or-&quot;many-to-many&quot;-transfer-modes" data-toc-modified-id="Scaling-up-transfer-by-the-&quot;one-to-many&quot;,-&quot;many-to-one&quot;,-or-&quot;many-to-many&quot;-transfer-modes-9.3"><span class="toc-item-num">9.3&nbsp;&nbsp;</span>Scaling up transfer by the "one-to-many", "many-to-one", or "many-to-many" transfer modes</a></span></li></ul></li><li><span><a href="#Scaling-up-transfers-using-a-for-loop" data-toc-modified-id="Scaling-up-transfers-using-a-for-loop-10"><span class="toc-item-num">10&nbsp;&nbsp;</span>Scaling up transfers using a for-loop</a></span><ul class="toc-item"><li><span><a href="#Nested-for-loops-example" data-toc-modified-id="Nested-for-loops-example-10.1"><span class="toc-item-num">10.1&nbsp;&nbsp;</span>Nested for-loops example</a></span></li></ul></li><li><span><a href="#Note-on-using-8-channel-pipettes" data-toc-modified-id="Note-on-using-8-channel-pipettes-11"><span class="toc-item-num">11&nbsp;&nbsp;</span>Note on using 8-channel pipettes</a></span></li><li><span><a href="#Configuring-the-script-for-proper-simulation-and-run" data-toc-modified-id="Configuring-the-script-for-proper-simulation-and-run-12"><span class="toc-item-num">12&nbsp;&nbsp;</span>Configuring the script for proper simulation and run</a></span></li><li><span><a href="#Proper-simulation-and-output-commands-to-an-external-log-file-for-inspection" data-toc-modified-id="Proper-simulation-and-output-commands-to-an-external-log-file-for-inspection-13"><span class="toc-item-num">13&nbsp;&nbsp;</span>Proper simulation and output commands to an external log file for inspection</a></span></li><li><span><a href="#Customizing-the-liquid-transfer-process" data-toc-modified-id="Customizing-the-liquid-transfer-process-14"><span class="toc-item-num">14&nbsp;&nbsp;</span>Customizing the liquid transfer process</a></span><ul class="toc-item"><li><span><a href="#Pipetting-speed" data-toc-modified-id="Pipetting-speed-14.1"><span class="toc-item-num">14.1&nbsp;&nbsp;</span>Pipetting speed</a></span></li><li><span><a href="#Well-location" data-toc-modified-id="Well-location-14.2"><span class="toc-item-num">14.2&nbsp;&nbsp;</span>Well location</a></span></li><li><span><a href="#Customization-on-Transfers" data-toc-modified-id="Customization-on-Transfers-14.3"><span class="toc-item-num">14.3&nbsp;&nbsp;</span>Customization on Transfers</a></span></li></ul></li><li><span><a href="#Achieving-fine-control-through-&quot;building-block&quot;-commands" data-toc-modified-id="Achieving-fine-control-through-&quot;building-block&quot;-commands-15"><span class="toc-item-num">15&nbsp;&nbsp;</span>Achieving fine control through "building block" commands</a></span><ul class="toc-item"><li><span><a href="#Example-1:-the-blow_out=True-issue" data-toc-modified-id="Example-1:-the-blow_out=True-issue-15.1"><span class="toc-item-num">15.1&nbsp;&nbsp;</span>Example 1: the blow_out=True issue</a></span></li><li><span><a href="#Example-2:-Delay-between-aspiration-and-dispense" data-toc-modified-id="Example-2:-Delay-between-aspiration-and-dispense-15.2"><span class="toc-item-num">15.2&nbsp;&nbsp;</span>Example 2: Delay between aspiration and dispense</a></span></li><li><span><a href="#Example-3:-Mixing-with-customized-speeds" data-toc-modified-id="Example-3:-Mixing-with-customized-speeds-15.3"><span class="toc-item-num">15.3&nbsp;&nbsp;</span>Example 3: Mixing with customized speeds</a></span></li></ul></li><li><span><a href="#Writing-reusable-scripts" data-toc-modified-id="Writing-reusable-scripts-16"><span class="toc-item-num">16&nbsp;&nbsp;</span>Writing reusable scripts</a></span></li><li><span><a href="#Manual-intergration-with-Excel" data-toc-modified-id="Manual-intergration-with-Excel-17"><span class="toc-item-num">17&nbsp;&nbsp;</span>Manual intergration with Excel</a></span></li><li><span><a href="#Additional-remarks" data-toc-modified-id="Additional-remarks-18"><span class="toc-item-num">18&nbsp;&nbsp;</span>Additional remarks</a></span></li></ul></div>

# Introduction
This workshop aims at making the OT-2 APIv2 as approachable as possible for people like me who have limited experience in programming. The workshop material below is in someway just a skimmed walkthrough of the [Opentrons OT-2 APIv2 docs](http://docs.opentrons.com/v2). However, this is not meant to be a OT-2 programming 101 course and so it does not follow the official order nor their recommendations. OT-2 users who wish to make the most out of the robot are encouraged to peruse the docs.

In the rest of the document, API refers to APIv2.

To summarize, our approach to the API is:

**First you make it work, then you make it work better**

Part 1: The basics, for start to finish

1. Build a virtual environment (a sandbox) to play around with features of the API  
2. Unit test the labwares, wells, pipettes and commands and get instant readouts
3. Generate a human-trackable unit action, i.e. one single liquid transfer
4. Loop that unit action to scale up the experiment
5. Simulate the draft in the virtual environment, get the simulated output and check whether it is correct
6. Remove the virtual environment and convert the draft into a proper script
7. Do a final confirmation by simulating the script and exporting its formal runlog (**important for experiment documentation**)

Part 2: Customizations, how to make our experiments really work, and how to make our lives easier

1. Customize the transfer process
2. Use building block commands to achieve control precision
3. Write reusable scripts to avoid creating new scripts from scratch every time
4. Semi-integrate with Microsoft Excel so outputs from other instruments could be used to direct transfer processes

# Disclaimer
1. I have limited experience in programming and received little formal training. Coding practices may be unorthodox and even heretic in the eyes of properly trained programmers.
2. Methods of execution are in most cases suboptimal in performance. (see Additional Remarks #1)
3. Please forgive my ocassional loss of professionalism in language.


# Intended Outcomes

By the end of the workshop, we hope participants would be able to:

1. Read the documentation on APIv2, and know where to look for more information, and where to seek help if needed
2. Set up a virtual environment for drafting a Python protocol for OT-2 APIv2
3. Debug the code using the variable explorer and the console
4. Import the opentrons library to initialize a protocol object, create labware objects and pipettes within it and retrieve their information
5. Access wells of plates / racks
6. Instruct the pipettes to carry out liquid movement using pipette.transfer() and pipette.distribute(), and configure them with additional parameters
7. Create a custom pipetting step using building block commands
8. Use loops, lists and dictionaries together to scale up pipetting steps
9. Simulate the protocol in a PC and then converting it to a robot-executable one
10. Export simulate results to a readable-file.
11. Write the framework of practically useful protocols for automatable experiments 
12. Use Excel to create list of parsable commands to be copied and pasted into a protocol

# Preparation: System Setup

## Install the IDE
It is best to make use of an Integrated Development Environment (IDE) when writing codes. My personal experience suggests that  Spyder is most friendly to Python beginners and its Variable Explorer is tremendously helpful, both in debugging and in understanding how the API works. Some other workshops recommend Jupyter or PyCharm. Participants are welcomed to stick with the IDEs they are most comfortable with.

The relatively fool-proof way to get Spyder is to download and install [Anaconda](https://www.anaconda.com/distribution/). For advanced users, the accompanying site packages like pandas and NumPy comes in handy for interfacing experiment results and experiment automation. (This will not be covered here)

## Install the opentrons module

Once Anaconda is installed, participants should open Spyder, and install the opentrons module / library, by typing in the IPython console:  

`pip install opentrons`

To check if the opentrons is correctly installed, participants should try and see if they could get the module version from the  console.

In [1]:
import opentrons
print(opentrons.__version__)

3.18.0


Failure to do so should give an error message of `ModuleNotFoundError: No module named 'opentrons'`

## Keeping the opentrons module up-to-date
Opentrons update their API and library from time to time and it should happen together with OT-2 App and robot firmware upgrade. Always check if the opentrons module is the latest, and if needed, upgrade the opentrons module on the computer by typing in the terminal  

`pip install opentrons --upgrade`

# Setting up a virtual environment for simulating a robot

The following section corresponds to ["Simulating your scripts"](https://docs.opentrons.com/v2/writing.html#simulating-your-scripts) from the API docs.

Let's start by typing the following into the Spyder Editor.

In [2]:
from opentrons import simulate
protocol = simulate.get_protocol_api(version = '2.4')

C:\Users\User\.opentrons\deck_calibration.json not found. Loading defaults
C:\Users\User\.opentrons\robot_settings.json not found. Loading defaults


Loading json containers...
Json container file load complete, listing database
Found 0 containers to add. Starting migration...
Database migration complete!


The `protocol` object is the starting point of all robot related actions, like choosing the pipette and putting labwares on deck. It also stores the simulated output of the robot. To inspect its functions and attributes, run `dir(protocol)` in the IPython console.

# Labware objects

The following section corresponds to ["Labware"](https://docs.opentrons.com/v2/new_labware.html) from the Opentrons docs.

## Putting labware objects on the deck

Opentrons' definition of labware includes both the mult-well plates/racks/reserviors and the tip racks. They are loaded onto the robot via the function:  
`labware_name = protocol.load_labware(load_name, location)`  
- load_name: what is the labware, **as called by Opentrons**?  
To look up this information, go to the [Opentrons official labware page](https://labware.opentrons.com/).
In the following example we will use a [200 µL 96-well flat bottom plate](https://labware.opentrons.com/corning_96_wellplate_360ul_flat).  
The `load_name` of this labware is `corning_96_wellplate_360ul_flat`.  
See Additional Remarks X for using a generic versus specific `load_name`.  
  
  
- slot: which of the slot (1-11) are we placing it?  
This argument must be input as an integer.

In [3]:
plate = protocol.load_labware('corning_96_wellplate_360ul_flat', 1)

Now if we inspect the variable explorer we should see the an object of `plate` being created. Any labware objects that we load by this command exists as a virtual object in the simulated environment.

In [4]:
plate

Corning 96 Well Plate 360 µL Flat on 1

Again, we can double click on the object in the variable explorer or use `dir(plate)` to look at the attributes of this plate.  
  
We also need to put tips and tip racks onto the deck (say slot 2). So we will add and run the following lines.

In [5]:
tip_rack = protocol.load_labware('opentrons_96_tiprack_300ul', 2)

It should also appear in the variable explorer, can be checked just by typing the variable in the console.

In [6]:
tip_rack

Opentrons 96 Tip Rack 300 µL on 2

## Getting wells within a labware

Before adding the pipettes let's briefy talk about how to get the wells in a labware object. This applies to whatever labware we are using, whether it's a multi-well plate or a tube rack or even a tip rack.

The [Accessor Methods](https://docs.opentrons.com/v2/new_labware.html#accessor-methods) section on the Opentrons docs provide multiple ways to get the wells on labware, but we will only introduce two methods, which are usually sufficient to deal with most situations.

The conventional or user-firendly way: `labware.wells_by_name()[well_name]`

In [7]:
# get a single well
plate.wells_by_name()['A6']

A6 of Corning 96 Well Plate 360 µL Flat on 1

In [8]:
# get a single well and assign it to a variable
sample_well = plate.wells_by_name()['A6']
sample_well

A6 of Corning 96 Well Plate 360 µL Flat on 1

The other way is that get the wells by their **well indices**. This can come in handy in some other situations.  
  
Method: `labware.wells()[well_index]`

Well index for a 96-well format:


|   | 1 | 2  | 3  | 4  | 5  | 6  | 7  | 8  | 9  | 10 | 11 | 12 |
|---|---|----|----|----|----|----|----|----|----|----|----|----|
| A | 0 | 8  | 16 | 24 | 32 | 40 | 48 | 56 | 64 | 72 | 80 | 88 |
| B | 1 | 9  | 17 | 25 | 33 | 41 | 49 | 57 | 65 | 73 | 81 | 89 |
| C | 2 | 10 | 18 | 26 | 34 | 42 | 50 | 58 | 66 | 74 | 82 | 90 |
| D | 3 | 11 | 19 | 27 | 35 | 43 | 51 | 59 | 67 | 75 | 83 | 91 |
| E | 4 | 12 | 20 | 28 | 36 | 44 | 52 | 60 | 68 | 76 | 84 | 92 |
| F | 5 | 13 | 21 | 29 | 37 | 45 | 53 | 61 | 69 | 77 | 85 | 93 |
| G | 6 | 14 | 22 | 30 | 38 | 46 | 54 | 62 | 70 | 78 | 86 | 94 |
| H | 7 | 15 | 23 | 31 | 39 | 47 | 55 | 63 | 71 | 79 | 87 | 95 |

Well index for a 24-well format (whether tube rack or 24-well plate):

|   | 1 | 2 | 3  | 4  | 5  | 6  |
|---|---|---|----|----|----|----|
| A | 0 | 4 | 8  | 12 | 16 | 20 |
| B | 1 | 5 | 9  | 13 | 17 | 21 |
| C | 2 | 6 | 10 | 14 | 18 | 22 |
| D | 3 | 7 | 11 | 15 | 19 | 23 |

In [9]:
# Get well A6 in 96-well plate
plate.wells()[40]

A6 of Corning 96 Well Plate 360 µL Flat on 1

In [10]:
# Get well A6 in 24-well plate
plate2 = protocol.load_labware('corning_24_wellplate_3.4ml_flat', 3)
plate2.wells()[20]

A6 of Corning 24 Well Plate 3.4 mL Flat on 3

One advantage of using well index instead of well name, is that we can easily get a **continuous** list of wells by inputing a "slice" of the list.

In [11]:
plate.wells()[0:12]

[A1 of Corning 96 Well Plate 360 µL Flat on 1,
 B1 of Corning 96 Well Plate 360 µL Flat on 1,
 C1 of Corning 96 Well Plate 360 µL Flat on 1,
 D1 of Corning 96 Well Plate 360 µL Flat on 1,
 E1 of Corning 96 Well Plate 360 µL Flat on 1,
 F1 of Corning 96 Well Plate 360 µL Flat on 1,
 G1 of Corning 96 Well Plate 360 µL Flat on 1,
 H1 of Corning 96 Well Plate 360 µL Flat on 1,
 A2 of Corning 96 Well Plate 360 µL Flat on 1,
 B2 of Corning 96 Well Plate 360 µL Flat on 1,
 C2 of Corning 96 Well Plate 360 µL Flat on 1,
 D2 of Corning 96 Well Plate 360 µL Flat on 1]

Note that we **cannot** use these methods to get mulitple specific wells

In [12]:
# Wrong way to get multiple wells by the .wells_by_name()
plate.wells_by_name()['A1', 'B2', 'C3']

KeyError: ('A1', 'B2', 'C3')

In [13]:
# Wrong way to get multiple wells by the .wells()
plate.wells()[1, 2, 3]

TypeError: list indices must be integers or slices, not tuple

# Setting up pipettes

The following section corresponds to ["Loading A Pipette"](https://docs.opentrons.com/v2/new_pipette.html#loading-a-pipette) from the API docs.

The way to set up a pipette is to call the function: `pipette_name = protocol.load_instrument(instrumnt_name, mount, tip_racks)`

Refer to the ["Pipette Models"](https://docs.opentrons.com/v2/new_pipette.html#pipette-models) for what instrument_name corresponds to what pipette.

Note 1: The argument `tip_racks` must be a **list**, this is a mistake I often make.

Note 2 : **Always** load the tip racks onto the virtual deck beforehand and **associate then those tip racks with the pipette** when we set up the pipette.

In [14]:
# load a P300S on the right mount

r_pipette = protocol.load_instrument(
    'p300_single', # instrument_name
    'right', # mount
    tip_racks=[tip_rack] # tip_racks as list, note that we are re-using the tip_rack object loaded before
)

r_pipette

<InstrumentContext: p300_single_v1 in RIGHT>

In [15]:
# Similarily, we can load a P10M on the left mount
# Remember to load a compatible tip rack first.

tip_rack_10 = protocol.load_labware('opentrons_96_tiprack_10ul', 4)

l_pipette = protocol.load_instrument(
    'p10_multi',
    'left',
    tip_racks=[tip_rack_10]
)

l_pipette

<InstrumentContext: p10_multi_v1 in LEFT>

For the following examples we will stick with the P300S on the right mount only.

# Getting readouts from the console

Basically by this time we have configured the robot properly. We can then instruct the pipette to take actions.  
  
Before we move on to introduce the pipette commands, we will first include the following lines at the end of the script:

In [16]:
print(*protocol.commands(), sep = '\n')

This line prints all the commands from the robot and will help us debug what is going on inside the script.

By this time, our script should look like the following:

In [50]:
from opentrons import simulate
protocol = simulate.get_protocol_api(version = '2.4')

#%% Robot set up

# we slightly rearranged the locations
plate1 = protocol.load_labware('corning_96_wellplate_360ul_flat', 1)
plate2 = protocol.load_labware('corning_24_wellplate_3.4ml_flat', 2)
tip_rack = protocol.load_labware('opentrons_96_tiprack_300ul', 3)

r_pipette = protocol.load_instrument(
    'p300_single', # instrument_name
    'right', # mount
    tip_racks=[tip_rack] # tip_racks as list, note that we are re-using the tip_rack object loaded before
)

#%% Robot actions

print(*protocol.commands(), sep = '\n')

C:\Users\User\.opentrons\deck_calibration.json not found. Loading defaults
C:\Users\User\.opentrons\robot_settings.json not found. Loading defaults


There is nothing from the console yet because we haven't assigned any commands to the pipettes.

# Executing a simple liquid transfer

The following section corresponds to ["Transfer"](https://docs.opentrons.com/v2/new_complex_commands.html#transfer) from the API docs, under the page ["Complex commands"](https://docs.opentrons.com/v2/new_complex_commands.html).

We will only introduce the `pipette.transfer()` command since it is most versatile. There is really no need to use `pipette.distribute()` or `pipette.consolidate()` since `pipette.transfer()` covers those functions already, unless it's for script readability.

The function should be called as follow:
`pipette.transfer(volume, source, dest)`

*Note: for this notebook the `print(protocol.commands())` function and the `protocol.clear_commands()` functions are called in every cell. This is not necessary when debugging the script in Spyder*

## "one-to-one" transfer

In [51]:
r_pipette.transfer(200, # volume in µL
                 plate2.wells_by_name()['A1'], # source well
                 plate1.wells_by_name()['A1'] # dest well
                )

print(*protocol.commands(), sep = '\n')

Transferring 200.0 from A1 of Corning 24 Well Plate 3.4 mL Flat on 2 to A1 of Corning 96 Well Plate 360 µL Flat on 1
Picking up tip from A1 of Opentrons 96 Tip Rack 300 µL on 3
Aspirating 200.0 uL from A1 of Corning 24 Well Plate 3.4 mL Flat on 2 at 1.0 speed
Dispensing 200.0 uL into A1 of Corning 96 Well Plate 360 µL Flat on 1 at 1.0 speed
Dropping tip into A1 of Opentrons Fixed Trash on 12


The output lines above are the instructions that the robot will execute, written in a human readable format. This is the log of the simuation and we should use that to check whether we wrote the script correctly.  
  
The first line "Transferring ... Flat on 1" is actually a section header that encapsulates all the subcommands (or building block commands) that follow it. We will introduce building commands later.

We can run another transfer step and pipette another 200 µL, but this time to well A2 on the 96-well plate.

In [52]:
# a second transfer step

r_pipette.transfer(200, # volume in µL
                 plate2.wells_by_name()['A1'], # source well
                 plate1.wells_by_name()['A2'] # dest well
                )

print(*protocol.commands(), sep = '\n')
protocol.clear_commands()

Transferring 200.0 from A1 of Corning 24 Well Plate 3.4 mL Flat on 2 to A1 of Corning 96 Well Plate 360 µL Flat on 1
Picking up tip from A1 of Opentrons 96 Tip Rack 300 µL on 3
Aspirating 200.0 uL from A1 of Corning 24 Well Plate 3.4 mL Flat on 2 at 1.0 speed
Dispensing 200.0 uL into A1 of Corning 96 Well Plate 360 µL Flat on 1 at 1.0 speed
Dropping tip into A1 of Opentrons Fixed Trash on 12
Transferring 200.0 from A1 of Corning 24 Well Plate 3.4 mL Flat on 2 to A2 of Corning 96 Well Plate 360 µL Flat on 1
Picking up tip from B1 of Opentrons 96 Tip Rack 300 µL on 3
Aspirating 200.0 uL from A1 of Corning 24 Well Plate 3.4 mL Flat on 2 at 1.0 speed
Dispensing 200.0 uL into A2 of Corning 96 Well Plate 360 µL Flat on 1 at 1.0 speed
Dropping tip into A1 of Opentrons Fixed Trash on 12


Reading the output above we take note of two things:

1. Two output commands are displayed instead of one. This is because the robot keeps on remembering new instructions that we have given it. This is usually not a problem when debugging in Spyder but one should always watch out for unintended duplciation of pipetting steps while writing the script.  
  
  
2. We notice that in the 1st step, the pipette picks the tip from position A1 of the rack and in the 2nd step, it does so from position B1. This is the perk of associating a tip rack to the pipette during the `load_instrument()` function, i.e., the robot automatically keep tracks of tip usage (column by column from A1 to A8, B1 to B8...)

## Large volumes

If we need to transfer, say, 400 µL but we only have a P300S, the `pipette.transfer()` function automatically does the calculation and performs two transfer steps of 200 µL each.

In [55]:
r_pipette.transfer(400, # volume in µL
                 plate2.wells_by_name()['A1'], # source well
                 plate1.wells_by_name()['A2'] # dest well
                )

print(*protocol.commands(), sep = '\n')
protocol.clear_commands()

Transferring 400.0 from A1 of Corning 24 Well Plate 3.4 mL Flat on 2 to A2 of Corning 96 Well Plate 360 µL Flat on 1
Picking up tip from G1 of Opentrons 96 Tip Rack 300 µL on 3
Aspirating 200.0 uL from A1 of Corning 24 Well Plate 3.4 mL Flat on 2 at 1.0 speed
Dispensing 200.0 uL into A2 of Corning 96 Well Plate 360 µL Flat on 1 at 1.0 speed
Aspirating 200.0 uL from A1 of Corning 24 Well Plate 3.4 mL Flat on 2 at 1.0 speed
Dispensing 200.0 uL into A2 of Corning 96 Well Plate 360 µL Flat on 1 at 1.0 speed
Dropping tip into A1 of Opentrons Fixed Trash on 12


## Scaling up transfer by the "one-to-many", "many-to-one", or "many-to-many" transfer modes

The `pipette.transfer()` function is extremely versatile. See the following examples.

In [58]:
# one-to-many
r_pipette.transfer(200,
                 plate2.wells_by_name()['A1'],
                 [
                     plate1.wells_by_name()['A1'],
                     plate1.wells_by_name()['A2'],
                     plate1.wells_by_name()['A3'] # instead of a single well, pass a list of destination wells
                 ] # notice where the open and close brackets are
                )

print(*protocol.commands(), sep = '\n')
protocol.clear_commands()

Transferring 200.0 from A1 of Corning 24 Well Plate 3.4 mL Flat on 2 to A1 of Corning 96 Well Plate 360 µL Flat on 1
Picking up tip from B2 of Opentrons 96 Tip Rack 300 µL on 3
Aspirating 200.0 uL from A1 of Corning 24 Well Plate 3.4 mL Flat on 2 at 1.0 speed
Dispensing 200.0 uL into A1 of Corning 96 Well Plate 360 µL Flat on 1 at 1.0 speed
Aspirating 200.0 uL from A1 of Corning 24 Well Plate 3.4 mL Flat on 2 at 1.0 speed
Dispensing 200.0 uL into A2 of Corning 96 Well Plate 360 µL Flat on 1 at 1.0 speed
Aspirating 200.0 uL from A1 of Corning 24 Well Plate 3.4 mL Flat on 2 at 1.0 speed
Dispensing 200.0 uL into A3 of Corning 96 Well Plate 360 µL Flat on 1 at 1.0 speed
Dropping tip into A1 of Opentrons Fixed Trash on 12


In [60]:
# many-to-one

r_pipette.transfer(200,
                 [
                     plate2.wells_by_name()['A1'],
                     plate2.wells_by_name()['A2'],
                     plate2.wells_by_name()['A3'] # instead of a single well, pass a list of destination wells
                 ], # again, notice where the open and close brackets are
                plate1.wells_by_name()['A1']
                )

print(*protocol.commands(), sep = '\n')
protocol.clear_commands()

Transferring 200.0 from A1 of Corning 24 Well Plate 3.4 mL Flat on 2 to A1 of Corning 96 Well Plate 360 µL Flat on 1
Picking up tip from D2 of Opentrons 96 Tip Rack 300 µL on 3
Aspirating 200.0 uL from A1 of Corning 24 Well Plate 3.4 mL Flat on 2 at 1.0 speed
Dispensing 200.0 uL into A1 of Corning 96 Well Plate 360 µL Flat on 1 at 1.0 speed
Aspirating 200.0 uL from A2 of Corning 24 Well Plate 3.4 mL Flat on 2 at 1.0 speed
Dispensing 200.0 uL into A1 of Corning 96 Well Plate 360 µL Flat on 1 at 1.0 speed
Aspirating 200.0 uL from A3 of Corning 24 Well Plate 3.4 mL Flat on 2 at 1.0 speed
Dispensing 200.0 uL into A1 of Corning 96 Well Plate 360 µL Flat on 1 at 1.0 speed
Dropping tip into A1 of Opentrons Fixed Trash on 12


In [62]:
# many-to-many
# A1 -> B1, A2 -> B2, A3 -> B3,

r_pipette.transfer(200,
                 [
                     plate1.wells_by_name()['A1'],
                     plate1.wells_by_name()['A2'],
                     plate1.wells_by_name()['A3'] # a list of source wells
                 ],
                 [
                     plate1.wells_by_name()['B1'],
                     plate1.wells_by_name()['B2'],
                     plate1.wells_by_name()['B3'] # a list of destination wells
                 ]
                )

print(*protocol.commands(), sep = '\n')
protocol.clear_commands()

Transferring 200.0 from A1 of Corning 96 Well Plate 360 µL Flat on 1 to B1 of Corning 96 Well Plate 360 µL Flat on 1
Picking up tip from F2 of Opentrons 96 Tip Rack 300 µL on 3
Aspirating 200.0 uL from A1 of Corning 96 Well Plate 360 µL Flat on 1 at 1.0 speed
Dispensing 200.0 uL into B1 of Corning 96 Well Plate 360 µL Flat on 1 at 1.0 speed
Aspirating 200.0 uL from A2 of Corning 96 Well Plate 360 µL Flat on 1 at 1.0 speed
Dispensing 200.0 uL into B2 of Corning 96 Well Plate 360 µL Flat on 1 at 1.0 speed
Aspirating 200.0 uL from A3 of Corning 96 Well Plate 360 µL Flat on 1 at 1.0 speed
Dispensing 200.0 uL into B3 of Corning 96 Well Plate 360 µL Flat on 1 at 1.0 speed
Dropping tip into A1 of Opentrons Fixed Trash on 12


When using many-to-many transfer mode, make sure that we have **equal numbers of source and destination wells**.

In [63]:
# many-to-many, failed example
# A1 -> B1, A2 -> B2, A3 -> ?,

r_pipette.transfer(200,
                 [
                     plate1.wells_by_name()['A1'],
                     plate1.wells_by_name()['A2'],
                     plate1.wells_by_name()['A3'] # 3 source wells
                 ],
                 [
                     plate1.wells_by_name()['B1'],
                     plate1.wells_by_name()['B2'] # but only 2 destination wells
                 ]
                )

print(*protocol.commands(), sep = '\n')
protocol.clear_commands()

# Look at error below:

ValueError: Source and destination lists must be divisible

In [69]:
# This cell resets the robot from the error, this is not required in Spyder

from opentrons import simulate
protocol = simulate.get_protocol_api(version = '2.4')

#%% Robot set up

# we slightly rearranged the locations
plate1 = protocol.load_labware('corning_96_wellplate_360ul_flat', 1)
plate2 = protocol.load_labware('corning_24_wellplate_3.4ml_flat', 2)
tip_rack = protocol.load_labware('opentrons_96_tiprack_300ul', 3)

r_pipette = protocol.load_instrument(
    'p300_single', # instrument_name
    'right', # mount
    tip_racks=[tip_rack] # tip_racks as list
)

C:\Users\User\.opentrons\deck_calibration.json not found. Loading defaults
C:\Users\User\.opentrons\robot_settings.json not found. Loading defaults


If we arrange the wells such that they are continuous along columns, using well indices would come in really handy.

In [70]:
# many-to-many, along a column
# A1 -> A2,
# B1 -> B2,
# C1 -> C2,
# D1 -> D2

r_pipette.transfer(200,
                   plate1.wells()[0:4],
                   plate1.wells()[8:12]
                  )

print(*protocol.commands(), sep = '\n')
protocol.clear_commands()

Transferring 200.0 from A1 of Corning 96 Well Plate 360 µL Flat on 1 to A2 of Corning 96 Well Plate 360 µL Flat on 1
Picking up tip from A1 of Opentrons 96 Tip Rack 300 µL on 3
Aspirating 200.0 uL from A1 of Corning 96 Well Plate 360 µL Flat on 1 at 1.0 speed
Dispensing 200.0 uL into A2 of Corning 96 Well Plate 360 µL Flat on 1 at 1.0 speed
Aspirating 200.0 uL from B1 of Corning 96 Well Plate 360 µL Flat on 1 at 1.0 speed
Dispensing 200.0 uL into B2 of Corning 96 Well Plate 360 µL Flat on 1 at 1.0 speed
Aspirating 200.0 uL from C1 of Corning 96 Well Plate 360 µL Flat on 1 at 1.0 speed
Dispensing 200.0 uL into C2 of Corning 96 Well Plate 360 µL Flat on 1 at 1.0 speed
Aspirating 200.0 uL from D1 of Corning 96 Well Plate 360 µL Flat on 1 at 1.0 speed
Dispensing 200.0 uL into D2 of Corning 96 Well Plate 360 µL Flat on 1 at 1.0 speed
Dropping tip into A1 of Opentrons Fixed Trash on 12


# Scaling up transfers using a for-loop

We can now repeat a single one-to-one transfer step over and over again. This is the biggest advantage of programming through the API, compared to the Protocol Designer.

In [71]:
protocol.clear_commands()

for i in range(3):
    r_pipette.transfer(200, # volume in µL
                     plate2.wells_by_name()['A1'], # source well
                     plate1.wells_by_name()['A2'] # dest well
                    )

print(*protocol.commands(), sep = '\n')

Transferring 200.0 from A1 of Corning 24 Well Plate 3.4 mL Flat on 2 to A2 of Corning 96 Well Plate 360 µL Flat on 1
Picking up tip from B1 of Opentrons 96 Tip Rack 300 µL on 3
Aspirating 200.0 uL from A1 of Corning 24 Well Plate 3.4 mL Flat on 2 at 1.0 speed
Dispensing 200.0 uL into A2 of Corning 96 Well Plate 360 µL Flat on 1 at 1.0 speed
Dropping tip into A1 of Opentrons Fixed Trash on 12
Transferring 200.0 from A1 of Corning 24 Well Plate 3.4 mL Flat on 2 to A2 of Corning 96 Well Plate 360 µL Flat on 1
Picking up tip from C1 of Opentrons 96 Tip Rack 300 µL on 3
Aspirating 200.0 uL from A1 of Corning 24 Well Plate 3.4 mL Flat on 2 at 1.0 speed
Dispensing 200.0 uL into A2 of Corning 96 Well Plate 360 µL Flat on 1 at 1.0 speed
Dropping tip into A1 of Opentrons Fixed Trash on 12
Transferring 200.0 from A1 of Corning 24 Well Plate 3.4 mL Flat on 2 to A2 of Corning 96 Well Plate 360 µL Flat on 1
Picking up tip from D1 of Opentrons 96 Tip Rack 300 µL on 3
Aspirating 200.0 uL from A1 of Co

We can see that the identical action is repeated three times but every time a new tip is used.  
  
Repeating the identical step over and over again isn't very useful, however. Very often we want to do a one-to-many or many-to-many transfer. We can modify the scripts above to do the following:

In [76]:
# one-to-many, essentially a pipette.distribute() process
# A1 -> B1, B2, B3, B4

protocol.clear_commands()

dest_wells = ['B1', 'B2', 'B3', 'B4']

for dest_well in dest_wells:
    r_pipette.transfer(200,
                     plate1.wells_by_name()['A1'],
                     plate1.wells_by_name()[ dest_well ] # dynamic dest well that changes on every loop
                    )

print(*protocol.commands(), sep = '\n')

Transferring 200.0 from A1 of Corning 96 Well Plate 360 µL Flat on 1 to B1 of Corning 96 Well Plate 360 µL Flat on 1
Picking up tip from A2 of Opentrons 96 Tip Rack 300 µL on 3
Aspirating 200.0 uL from A1 of Corning 96 Well Plate 360 µL Flat on 1 at 1.0 speed
Dispensing 200.0 uL into B1 of Corning 96 Well Plate 360 µL Flat on 1 at 1.0 speed
Dropping tip into A1 of Opentrons Fixed Trash on 12
Transferring 200.0 from A1 of Corning 96 Well Plate 360 µL Flat on 1 to B2 of Corning 96 Well Plate 360 µL Flat on 1
Picking up tip from B2 of Opentrons 96 Tip Rack 300 µL on 3
Aspirating 200.0 uL from A1 of Corning 96 Well Plate 360 µL Flat on 1 at 1.0 speed
Dispensing 200.0 uL into B2 of Corning 96 Well Plate 360 µL Flat on 1 at 1.0 speed
Dropping tip into A1 of Opentrons Fixed Trash on 12
Transferring 200.0 from A1 of Corning 96 Well Plate 360 µL Flat on 1 to B3 of Corning 96 Well Plate 360 µL Flat on 1
Picking up tip from C2 of Opentrons 96 Tip Rack 300 µL on 3
Aspirating 200.0 uL from A1 of Co

In [77]:
# many-to-many
# A1 -> B1,
# A2 -> B2,
# A3 -> B3,
# A4 -> B4

protocol.clear_commands()

source_wells = ['A1', 'A2', 'A3', 'A4']
dest_wells = ['B1', 'B2', 'B3', 'B4']

for i in range(4):
    r_pipette.transfer(200,
                     plate1.wells_by_name()[ source_wells[i] ], # dynamic source well that changes on every loop
                     plate1.wells_by_name()[ dest_wells[i] ] # dynamic dest well that changes on every loop
                    )

print(*protocol.commands(), sep = '\n')

Transferring 200.0 from A1 of Corning 96 Well Plate 360 µL Flat on 1 to B1 of Corning 96 Well Plate 360 µL Flat on 1
Picking up tip from E2 of Opentrons 96 Tip Rack 300 µL on 3
Aspirating 200.0 uL from A1 of Corning 96 Well Plate 360 µL Flat on 1 at 1.0 speed
Dispensing 200.0 uL into B1 of Corning 96 Well Plate 360 µL Flat on 1 at 1.0 speed
Dropping tip into A1 of Opentrons Fixed Trash on 12
Transferring 200.0 from A2 of Corning 96 Well Plate 360 µL Flat on 1 to B2 of Corning 96 Well Plate 360 µL Flat on 1
Picking up tip from F2 of Opentrons 96 Tip Rack 300 µL on 3
Aspirating 200.0 uL from A2 of Corning 96 Well Plate 360 µL Flat on 1 at 1.0 speed
Dispensing 200.0 uL into B2 of Corning 96 Well Plate 360 µL Flat on 1 at 1.0 speed
Dropping tip into A1 of Opentrons Fixed Trash on 12
Transferring 200.0 from A3 of Corning 96 Well Plate 360 µL Flat on 1 to B3 of Corning 96 Well Plate 360 µL Flat on 1
Picking up tip from G2 of Opentrons 96 Tip Rack 300 µL on 3
Aspirating 200.0 uL from A3 of Co

## Nested for-loops example

The above examples are equivalents of the `pipette.transfer()` function, but there is more that we can achieve with for-loops.

Consider the following example. We wish to pipette:

plate 1 well A1 to plate 1 well A1,
plate 1 well A2 to plate 1 well A2,
plate 1 well A3 to plate 1 well A3,
plate 1 well A4 to plate 1 well A4,

and then repeat the same process but substituting plate 1 with plate 2.

In [78]:
# many-to-many,
# for plate 1 and plate 2
# A1 -> B1,
# A2 -> B2,
# A3 -> B3,
# A4 -> B4

protocol.clear_commands()

source_wells = ['A1', 'A2', 'A3', 'A4']
dest_wells = ['B1', 'B2', 'B3', 'B4']

for plate in [plate1, plate2]:
    
    # the for-loop beneath will run twice
    for i in range(4):
        r_pipette.transfer(200,
                         plate.wells_by_name()[ source_wells[i] ],
                           # note that it is now a variable "plate" instead of the  object "plate1"
                         plate.wells_by_name()[ dest_wells[i] ] 
                        )

print(*protocol.commands(), sep = '\n')

Transferring 200.0 from A1 of Corning 96 Well Plate 360 µL Flat on 1 to B1 of Corning 96 Well Plate 360 µL Flat on 1
Picking up tip from A3 of Opentrons 96 Tip Rack 300 µL on 3
Aspirating 200.0 uL from A1 of Corning 96 Well Plate 360 µL Flat on 1 at 1.0 speed
Dispensing 200.0 uL into B1 of Corning 96 Well Plate 360 µL Flat on 1 at 1.0 speed
Dropping tip into A1 of Opentrons Fixed Trash on 12
Transferring 200.0 from A2 of Corning 96 Well Plate 360 µL Flat on 1 to B2 of Corning 96 Well Plate 360 µL Flat on 1
Picking up tip from B3 of Opentrons 96 Tip Rack 300 µL on 3
Aspirating 200.0 uL from A2 of Corning 96 Well Plate 360 µL Flat on 1 at 1.0 speed
Dispensing 200.0 uL into B2 of Corning 96 Well Plate 360 µL Flat on 1 at 1.0 speed
Dropping tip into A1 of Opentrons Fixed Trash on 12
Transferring 200.0 from A3 of Corning 96 Well Plate 360 µL Flat on 1 to B3 of Corning 96 Well Plate 360 µL Flat on 1
Picking up tip from C3 of Opentrons 96 Tip Rack 300 µL on 3
Aspirating 200.0 uL from A3 of Co

# Note on using 8-channel pipettes

8-channel pipettes deal with 8 wells at the same time so for a `pipette.transfer` we can pass in a list of 8 well objects as the argument for `location`. We can also just assume the 8-channel pipette as a single channel pipette and use the top channel as the pointer.

Example:

In [89]:
# Alternative 1: specifying 8 wells.

from opentrons import simulate
protocol = simulate.get_protocol_api(version = '2.4')

plate1 = protocol.load_labware('corning_96_wellplate_360ul_flat', 1)
tip_rack = protocol.load_labware('opentrons_96_tiprack_300ul', 2)

r_pipette = protocol.load_instrument(
    'p300_multi', # we use a 8-channel pipette here
    'right',
    tip_racks=[tip_rack]
)
r_pipette.transfer(200,
                   plate1.wells()[0:8], # wells A1-H1, or we can use plate1.columns_by_name()['1']
                   plate1.wells()[8:16] # wells A2-H2, or we can use plate1.columns_by_name()['2']
                      )
print(*protocol.commands(), sep = '\n')

C:\Users\User\.opentrons\deck_calibration.json not found. Loading defaults
C:\Users\User\.opentrons\robot_settings.json not found. Loading defaults


Transferring 200.0 from A1 of Corning 96 Well Plate 360 µL Flat on 1 to A2 of Corning 96 Well Plate 360 µL Flat on 1
Picking up tip from A1 of Opentrons 96 Tip Rack 300 µL on 3
Aspirating 200.0 uL from A1 of Corning 96 Well Plate 360 µL Flat on 1 at 1.0 speed
Dispensing 200.0 uL into A2 of Corning 96 Well Plate 360 µL Flat on 1 at 1.0 speed
Dropping tip into A1 of Opentrons Fixed Trash on 12


In [94]:
# Alternative 2: specifying the top well only

from opentrons import simulate
protocol = simulate.get_protocol_api(version = '2.4')

plate1 = protocol.load_labware('corning_96_wellplate_360ul_flat', 1)
tip_rack = protocol.load_labware('opentrons_96_tiprack_300ul', 2)

r_pipette = protocol.load_instrument(
    'p300_multi', # we use a 8-channel pipette here
    'right',
    tip_racks=[tip_rack]
)
r_pipette.transfer(200,
                   plate1.wells()[0], # well A1
                   plate1.wells()[8] # well A2
                  )
print(*protocol.commands(), sep = '\n')

C:\Users\User\.opentrons\deck_calibration.json not found. Loading defaults
C:\Users\User\.opentrons\robot_settings.json not found. Loading defaults


Transferring 200.0 from A1 of Corning 96 Well Plate 360 µL Flat on 1 to A2 of Corning 96 Well Plate 360 µL Flat on 1
Picking up tip from A1 of Opentrons 96 Tip Rack 300 µL on 2
Aspirating 200.0 uL from A1 of Corning 96 Well Plate 360 µL Flat on 1 at 1.0 speed
Dispensing 200.0 uL into A2 of Corning 96 Well Plate 360 µL Flat on 1 at 1.0 speed
Dropping tip into A1 of Opentrons Fixed Trash on 12


We can see that the there are no differences between the two methods. Usually, it is best to go for "specifying a single well". It works more relaibly in case you need to use a 8-channel pipette with a 384-well plate.

However, if the for-loop is used, we **must not loop through the groups of wells inside the `pipette.transfer()` function**

In [93]:
# Wrong example of for-loop together with 8-channel pipette.

from opentrons import simulate
protocol = simulate.get_protocol_api(version = '2.4')

plate1 = protocol.load_labware('corning_96_wellplate_360ul_flat', 1)
tip_rack = protocol.load_labware('opentrons_96_tiprack_300ul', 2)

r_pipette = protocol.load_instrument(
    'p300_multi', # we use a 8-channel pipette here
    'right',
    tip_racks=[tip_rack]
)

for i in [8, 9 ,10, 11, 12, 13, 14, 15, 16]:
    r_pipette.transfer(200,
                       plate1.wells()[0],
                       plate1.wells()[i] # the intention may be wells in column '2', but this 
                      )
    
print(*protocol.commands(), sep = '\n')

C:\Users\User\.opentrons\deck_calibration.json not found. Loading defaults
C:\Users\User\.opentrons\robot_settings.json not found. Loading defaults


RuntimeError: Invalid target for multichannel transfer: [B2 of Corning 96 Well Plate 360 µL Flat on 1]

# Configuring the script for proper simulation and run

# Proper simulation and output commands to an external log file for inspection

# Customizing the liquid transfer process

To have better controls over 

## Pipetting speed

rate = 
## Well location


## Customization on Transfers



# Achieving fine control through "building block" commands

It may appear that the `pipette.transfer()` function would suffice in many scenarios. In my personal experience, however, more often than not I would find unexpected behavior on the robot which jeopardize the entire experiment. For instance, mixing anything that is slighty more vicious than water, through capilliary action on the pipette tip, will lead to an incomplete dispense of liquid within each aspirate-dispense cycle. In the end, the tip used for transfer + mixing carried away a significant volume of liquid, and this could affect an experiment as simple as a serial dilution.

We will thus introduce the building block commands. Once we master them we will have much better control over the fine details of our pipetting steps. And this is in fact, surprisingly easy.

In [112]:
from opentrons import simulate
protocol = simulate.get_protocol_api(version = '2.4')

plate1 = protocol.load_labware('corning_96_wellplate_360ul_flat', 1)
plate2 = protocol.load_labware('corning_24_wellplate_3.4ml_flat', 2)
tip_rack = protocol.load_labware('opentrons_96_tiprack_300ul', 3)

r_pipette = protocol.load_instrument(
    'p300_single',
    'right',
    tip_racks=[tip_rack]
)

C:\Users\User\.opentrons\deck_calibration.json not found. Loading defaults
C:\Users\User\.opentrons\robot_settings.json not found. Loading defaults


Consider the following `pipette.transfer()` function

In [99]:
r_pipette.transfer(200, plate1.wells_by_name()['A1'], plate1.wells_by_name()['A2'])
print(*protocol.commands(), sep = '\n')
protocol.clear_commands()

Transferring 200.0 from A1 of Corning 96 Well Plate 360 µL Flat on 1 to A2 of Corning 96 Well Plate 360 µL Flat on 1
Picking up tip from A1 of Opentrons 96 Tip Rack 300 µL on 3
Aspirating 200.0 uL from A1 of Corning 96 Well Plate 360 µL Flat on 1 at 1.0 speed
Dispensing 200.0 uL into A2 of Corning 96 Well Plate 360 µL Flat on 1 at 1.0 speed
Dropping tip into A1 of Opentrons Fixed Trash on 12


If we rewrite the above by the building block commands:

In [106]:
r_pipette.pick_up_tip() # no argument required since the robot tracks tip usage for us
r_pipette.aspirate(200, # aspirate volume
                  plate1.wells_by_name()['A1'] # source well 
                  )
r_pipette.dispense(volume=r_pipette.current_volume,
                   location=plate1.wells_by_name()['A2'] # destination well
                  )
r_pipette.drop_tip()

print(*protocol.commands(), sep = '\n')
protocol.clear_commands()

Picking up tip from A1 of Opentrons 96 Tip Rack 300 µL on 3
Aspirating 200.0 uL from A1 of Corning 96 Well Plate 360 µL Flat on 1 at 1.0 speed
Dispensing 200.0 uL into A2 of Corning 96 Well Plate 360 µL Flat on 1 at 1.0 speed
Dropping tip into A1 of Opentrons Fixed Trash on 12


If we compare the outputs from `pipette.transfer()` and the building block commands, we can see that there is virtually no difference in how the robot executes the function. The only difference is that the encapsulation line "Transferring... ..." is no longer present.

Also note that we could have written `r_pipette.dispense(location=plate1.wells_by_name()['A2'])` directy instead.

So why should we bother using building block commands? Let's consider the following scenairo

In [107]:
r_pipette.transfer(200, plate1.wells_by_name()['A1'], plate1.wells_by_name()['A2'],
                  blow_out = True # Note: key change here
                  )
print(*protocol.commands(), sep = '\n')
protocol.clear_commands()

Transferring 200.0 from A1 of Corning 96 Well Plate 360 µL Flat on 1 to A2 of Corning 96 Well Plate 360 µL Flat on 1
Picking up tip from B1 of Opentrons 96 Tip Rack 300 µL on 3
Aspirating 200.0 uL from A1 of Corning 96 Well Plate 360 µL Flat on 1 at 1.0 speed
Dispensing 200.0 uL into A2 of Corning 96 Well Plate 360 µL Flat on 1 at 1.0 speed
Blowing out at A1 of Opentrons Fixed Trash on 12
Dropping tip into A1 of Opentrons Fixed Trash on 12


## Example 1: the blow_out=True issue

As it turns out, Opentrons decided that, if you wish to expunge all liquid from the tip, those liquid will go to the trash instead of your destination well!

As seasoned pipette users we all know it isn't how we usually do things. Alas, the logic in liquid handling automation is that you should always include a disposal volume during pipetting to increase pipetting accuracy. There will always be residual liquid sticking to the interior walls of the tip and we cannot afford to manually inspect them while the robot is running. So yes, there is a reason behind that.

However, we sometimes work with precious samples and sometimes, we just have to sequeez out every bit of liquid from the tip into the well. Assuming we have to do so, we can rewrite the single `pipette.transfer()` function in the following way:

In [109]:
r_pipette.pick_up_tip() 
r_pipette.aspirate(200,
                  plate1.wells_by_name()['A1']
                  )
r_pipette.dispense(volume=r_pipette.current_volume,
                   location=plate1.wells_by_name()['A2']
                  )
r_pipette.blow_out(location=plate1.wells_by_name()['A2']) # if no argument, blow out at current position 
r_pipette.drop_tip()

print(*protocol.commands(), sep = '\n')
protocol.clear_commands()

Picking up tip from D1 of Opentrons 96 Tip Rack 300 µL on 3
Aspirating 200.0 uL from A1 of Corning 96 Well Plate 360 µL Flat on 1 at 1.0 speed
Dispensing 200.0 uL into A2 of Corning 96 Well Plate 360 µL Flat on 1 at 1.0 speed
Blowing out at A2 of Corning 96 Well Plate 360 µL Flat on 1
Dropping tip into A1 of Opentrons Fixed Trash on 12


## Example 2: Delay between aspiration and dispense
In another instance, say we wish to wait 5 seconds after aspiration, to allow a slighlty viscous liquid to be properly aspirated before dispensing it elsewhere.

In [114]:
r_pipette.pick_up_tip() 
r_pipette.aspirate(200, plate1.wells_by_name()['A1'])

protocol.delay(seconds=5) # additional line

r_pipette.dispense(location=plate1.wells_by_name()['A2'])
r_pipette.drop_tip()

print(*protocol.commands(), sep = '\n')
protocol.clear_commands()

Picking up tip from B1 of Opentrons 96 Tip Rack 300 µL on 3
Aspirating 200.0 uL from A1 of Corning 96 Well Plate 360 µL Flat on 1 at 1.0 speed
Delaying for 0 minutes and 5 seconds
Dispensing 200.0 uL into A2 of Corning 96 Well Plate 360 µL Flat on 1 at 1.0 speed
Dropping tip into A1 of Opentrons Fixed Trash on 12


## Example 3: Mixing with customized speeds

One final example here is a highly customized mixing process for a peculiar need. Say, we wish to mix a sample 3 times, but:
1. The aspiration speed needs to be slower and the dispense speed needs to be faster.
2. We need to ensure all liquid is blown out before aspirating for the next cycle in mixing.
3. We want to dispense all liquid above the liquid level, not when the tip is still submerged inside the liquid.

This might look daunting, but with the building block commands it is in fact quite straightforward:

In [115]:
well = plate1.wells_by_name()['A1']

r_pipette.pick_up_tip() # use the same tip for mixing

for i in range(3):

    r_pipette.aspirate(150, # mix volume= 150 µL
                       well,
                       rate = .75 # slower rate for aspiration
                      )

    r_pipette.dispense(location=well.top(-5), # 2 mm from top of the well, ensures dispense above liquid level
                       rate = 1.5 # faster rate for dispense
                      )
    
r_pipette.drop_tip()

print(*protocol.commands(), sep = '\n')
protocol.clear_commands()

Picking up tip from C1 of Opentrons 96 Tip Rack 300 µL on 3
Aspirating 150.0 uL from A1 of Corning 96 Well Plate 360 µL Flat on 1 at 0.75 speed
Dispensing 150.0 uL into A1 of Corning 96 Well Plate 360 µL Flat on 1 at 1.5 speed
Aspirating 150.0 uL from A1 of Corning 96 Well Plate 360 µL Flat on 1 at 0.75 speed
Dispensing 150.0 uL into A1 of Corning 96 Well Plate 360 µL Flat on 1 at 1.5 speed
Aspirating 150.0 uL from A1 of Corning 96 Well Plate 360 µL Flat on 1 at 0.75 speed
Dispensing 150.0 uL into A1 of Corning 96 Well Plate 360 µL Flat on 1 at 1.5 speed
Dropping tip into A1 of Opentrons Fixed Trash on 12


We will notice that the exact location of the dispense step is not shown, but it will be carried out by the robot faithfully.

# Writing reusable scripts

# Manual intergration with Excel

# Additional remarks

1. It is faster to use `pipette.transfer()` with list of well objects than looping a a single `pipette.transfer()` function multiple times. The difference in speed in unnoticeable on a PC but could mean 4-5 minutes of wait when API of a long protocol is compiled on a Pi-powered robot.

The following example illustrates a comparison of the two methods, both trying to pipette 200 µL from well A1 to A7, B1 to B7, ... H6 to H12, for a total of 48 transfer steps.

A single complex function processes multi-to-multi transfers much more efficiently, at the expense of losing fine-tuned control at the atomic movement level.

In [18]:
# Set up
import time
from opentrons import simulate
protocol = simulate.get_protocol_api('2.2')
tip_rack = protocol.load_labware('opentrons_96_tiprack_300ul', location = '1')
plate = protocol.load_labware('corning_96_wellplate_360ul_flat', location = '2')
pipette = protocol.load_instrument('p300_single', mount = 'right', tip_racks = [tip_rack])
print('Run time comparison:')

# Method 1: List for looping pipette.transfer
transfer_info = {x: x + 48 for x in range(0,48)}

start_time = time.time()
for source_index, dest_index in transfer_info.items():
    pipette.transfer(200, plate.wells(source_index), plate.wells(dest_index))
end_time = time.time()
time_diff = end_time - start_time
print('Loop method: ' + str(time_diff))

# Method 2: Looping through a series of wells
source_wells = plate.columns()[0:3]
dest_wells = plate.columns()[4:7]
start_time = time.time()
pipette.transfer(200, source_wells, dest_wells)
end_time = time.time()
time_diff = end_time - start_time
print('Well object method: ' + str(time_diff))

C:\Users\User\.opentrons\deck_calibration.json not found. Loading defaults
C:\Users\User\.opentrons\robot_settings.json not found. Loading defaults


Run time comparison:
Loop method: 0.9275453090667725
Well object method: 0.16154098510742188


2. To interface with external files like .csv, the API provides a method but that is not recommended.
That is however not recommended.

3. Labware objects issues:
It is not important to have the exact labware for a labware that we do not have

4. When accessing wells on a labware, the direct access of wells without using a method is strongly discouraged. It might make it easier to read but when we start to build complex functions, it makes debugging much more difficult.

5. It is bad habit to share a tip rack between two pipettes